In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import threading
import time, sched
import schedule
import serial
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from lxml import etree

'''import Threading'''
import threading
from threading import Thread
from _thread import *
from queue import Queue
import subprocess

'''socket Library'''
import socket

import warnings
warnings.filterwarnings('ignore')

class Live_weather:
    def __init__(self):
        try:
            self.ob=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
            self.ob.bind(('192.168.1.118', 1024))
            self.ob.listen(5)
            thread2 = threading.Thread(target = self.client_server, daemon = True)
            thread2.start()
        except OSError:
            pass
#         self.driver_path = "C:/Users/Acer/Desktop/Weather/chromedriver_win32 (1)/chromedriver.exe"
        self.driver_path = "chromedriver.exe"
        self.service = Service(self.driver_path)
        self.driver = webdriver.Chrome(service = self.service)
        self.driver.implicitly_wait(10)
        self.url = "https://www.google.com/search?q=my+location+weather&sxsrf=APwXEdfOWZf-UuQKscV4dYtFs8sYb4vS4A%3A1681979049383&ei=qfZAZISCF_Py4-EPn72E6Ak&ved=0ahUKEwjE-oeehLj-AhVz-TgGHZ8eAZ0Q4dUDCA8&oq=my+location+weather&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQDEoECEEYAFAAWABgAGgAcAF4AIABAIgBAJIBAJgBAA&sclient=gws-wiz-serp"
        
        self.clients = set()
        self.clients_lock = threading.Lock()
        
    def aqi_data(self):
        self.driver_1 = webdriver.Chrome("C:/Users/Acer/Desktop/Weather/chromedriver_win32 (1)/chromedriver.exe")
        time.sleep(15)
        self.url_1 = "https://www.aqi.in/"
        
class google_weather(Live_weather):
    def weather(self):
        self.driver.get(self.url)
        self.driver.refresh()            
        time.sleep(5)
        self.temp = self.driver.find_element(By.XPATH, "//span[@class='wob_t q8U8x']").text
        self.ppt = self.driver.find_element(By.XPATH, "//span[@id='wob_pp']").text
        self.humidity = self.driver.find_element(By.XPATH, "//span[@id='wob_hm']").text
        self.desc = self.driver.find_element(By.XPATH, "//span[@id='wob_dc']").text
        time.sleep(1)
        self.wind_clk = self.driver.find_element(By.XPATH, "//span[@id='wob_ws']")
        self.wind_clk.click()
        time.sleep(1)
        try:
            self.wind_direction = self.driver.find_element(By.XPATH, '//*[@id="wind_image_0"]')
            self.wind_dir = self.wind_direction.get_attribute('alt')
        except NoSuchElementException:
            self.wind_direction = self.driver.find_element(By.XPATH, '//*[@id="wind_image_1"]')
            self.wind_dir = self.wind_direction.get_attribute('alt')
        time.sleep(2)
        self.driver.close()


class air_quality_data(Live_weather):
    def aqi_details(self):
        self.driver_1.implicitly_wait(10)
        self.driver_1.get(self.url_1)
        self.driver_1.fullscreen_window()
        time.sleep(60)
        try:
            self.aqi = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[1]/div/div[2]/div[1]/div")).text)
            self.PM_2_5 = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[2]/div/div/div[1]/span[1]")).text)
            self.PM_10 = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[2]/div/div/div[2]/span[1]")).text)
            self.SO2 = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[2]/div/div/div[3]/span[1]")).text)
            self.O3 = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[2]/div/div/div[4]/span[1]")).text)
            self.CO = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[2]/div/div/div[5]/span[1]")).text)
            self.NO2 = int((self.driver_1.find_element(By.XPATH, "/html/body/main/section[4]/div/div/div[4]/div/div/div[2]/div/div/div[6]/span[1]")).text)
        except ValueError:
            self.aqi=self.aqi
            self.PM_2_5=self.PM_2_5
            self.PM_10=self.PM_10
            self.SO2=self.SO2
            self.O3=self.O3
            self.CO=self.CO
            self.NO2=self.NO2
            pass
        self.driver_1.close()
        
class all_data(google_weather, air_quality_data):
    def send_data_to_serial_port(self):
        self.weather()
        data1 = str("Weather :--> ") + self.temp + str("°C") + str(", ") + self.desc
        self.Lan_comm(data1)
        time.sleep(7)
        data1 =  str("Humidity :--> ") + self.humidity
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("Precipitation :--> ") + self.ppt  
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("Wind Speed & direction :-->  ") + self.wind_dir
        self.Lan_comm(data1)
        self.aqi= self.aqi_thread()
        data1 = str("Air quality index :-->  ") + str(self.aqi)
        self.Lan_comm(data1)
        self.remark_and_impact(self.aqi)
        time.sleep(7)
        data1 = str("PM 2.5 is :-->  ") + str(self.PM_2_5)
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("PM 10 is :-->  ") + str(self.PM_10)
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("SO2 is :-->  ") + str(self.SO2)
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("O3 is :-->  ") + str(self.O3)
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("CO is :-->  ") + str(self.CO)
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("NO2 is :-->  ") + str(self.NO2)
        self.Lan_comm(data1)
        
    def remark_and_impact(self, aqi):
        if self.aqi<50:
            remark = 'Good'
            impact = "Minimal impact"
        elif 50<self.aqi<100:
            remark = "Satisfactory"
            impact = "Minor breathing discomfort to sensitive people"
        elif 100<self.aqi<200:
            remark= "Moderate"
            impact = "Breathing discomfort to the people with lungs, asthma and heart diseases"
        elif 200<self.aqi<300:
            remark= "Poor"
            impact = "Breathing discomfort to most people on prolonged exposure"
        elif 300<self.aqi<400:
            remark="Very Poor"
            impact = "Breathing discomfort to most people on prolonged exposure"
        elif 400<self.aqi<500:
            remark="Severe"
            impact = "Affects healthy people and seriously impacts those with existing diseases"
        time.sleep(7)
        data1 = str("remark :-->  ") + remark
        self.Lan_comm(data1)
        time.sleep(7)
        data1 = str("Impact :-->  ") + impact
        self.Lan_comm(data1)
        
    def aqi_thread(self):
        self.aqi_data()
        self.aqi_details()
        return self.aqi

    def Lan_comm(self, data1):
        print(data1)
        data2 = data1 + '\n'
        try:
            thread1 = threading.Thread(target = self.broadcasting_function, args=(data2,), daemon = True)
            thread1.start()
            try:
                if not connection:
                    thread2 = threading.Thread(target = self.client_server, daemon = True)
                    thread2.start()
                else:
                    pass
            except NameError:
                pass
        except OSError as error:
            pass
        
    def client_server(self):
        print("server is listening")
        try:
            while True:
                self.connection, self.addr = self.ob.accept()
        except OSError:
            pass

    def broadcasting_function(self, data2):
        try:
            try:
                with self.clients_lock:
                    self.clients.add(self.connection)
                self.conc = str(self.connection)
                if len(self.conc) > 140:
                    try:
                        while True:
                            if not data2:
                                break
                            else:
                                with self.clients_lock:
                                    for self.connections in self.clients:
                                        self.connections.sendall(str.encode(data2))
                                    data2=""
                    except ConnectionError:
                        if self.connection in self.clients:
                            self.clients.remove(self.connection)
                        return
                else:
                    if self.connection in self.clients:
                        self.clients.remove(self.connection)
                    self.connection=""
                    return
            except AttributeError:
                pass
        except NameError:
            pass
def main():
    obj3 = all_data()
    obj3.send_data_to_serial_port()
    
if __name__ == '__main__':
    main()
    
schedule.every(7).seconds.do(main)

while True:
    schedule.run_pending()
    time.sleep(1)  